In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
import seaborn as sns
sns.set()
sns.set_style("darkgrid")
import warnings
warnings.filterwarnings("ignore")
import math
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
import os
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
base_path = r'C:\Users\frank\OneDrive\Desktop\Machine Learning\MP#3'
path = os.path.join(base_path,'Data-Covid002.csv')
covid_data = pd.read_csv(path, encoding='ISO-8859-1')
path2 = os.path.join(base_path,'PPHA_30545_MP03-Variable_Description.xlsx')
data_dict = pd.read_excel(path2)

# Question 1

In [3]:
opp_insights_variables = data_dict.loc[data_dict['Source'] == 'Opportunity Insights', 'Variable']
pm_covid_variables = data_dict.loc[data_dict['Source'] == 'PM_COVID', 'Variable']
other_variables = ['county','state','deathspc']

In [4]:
opp_insights_df = covid_data[covid_data.columns.intersection(opp_insights_variables)]
pm_covid_df = covid_data[covid_data.columns.intersection(pm_covid_variables)]
other_variables_df = covid_data[covid_data.columns.intersection(other_variables)]

In [5]:
concat_list = [other_variables_df, opp_insights_df, pm_covid_df]
filtered_data = pd.concat(concat_list, axis='columns')
filtered_data.head(10)

,county,state,deathspc,intersects_msa,cur_smoke_q1,cur_smoke_q2,cur_smoke_q3,cur_smoke_q4,bmi_obese_q1,bmi_obese_q2,...,subcty_exp_pc,taxrate,tax_st_diff_top20,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax,pm25,bmcruderate,pm25_mia
0,Autauga,Alabama,16.548864,1,0.333333,0.238095,0.208333,0.133333,0.375000,0.238095,...,1059.66930,0.011183,0.0,306.02344,96.055420,288.08508,85.651848,11.712587,859.29999,0
1,Baldwin,Alabama,8.959118,1,0.268097,0.233503,0.167464,0.176991,0.298050,0.262467,...,2209.91040,0.011756,0.0,305.51663,97.971542,290.20886,89.730972,10.077723,976.20001,0
2,Barbour,Alabama,6.609756,0,0.228571,0.250000,0.181818,0.111111,0.294118,0.571429,...,1570.24830,0.012059,0.0,306.06226,97.371674,289.24210,88.633575,10.981967,1040.90000,0
3,Bibb,Alabama,6.038192,1,0.244444,0.280000,0.181818,0.150000,0.466667,0.375000,...,1338.44120,0.008007,0.0,305.98218,96.293076,287.36282,86.485870,11.998714,1028.80000,0
4,Blount,Alabama,1.503713,1,0.304348,0.260870,0.352941,0.166667,0.347826,0.318182,...,987.43884,0.007012,0.0,305.17886,94.630951,285.56567,85.449142,11.793022,993.70001,0
5,Bullock,Alabama,22.268559,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1858.17790,0.008011,0.0,305.86771,97.138420,288.69131,89.168121,11.313557,1166.60000,0
6,Butler,Alabama,69.481377,1,0.115385,0.136364,0.100000,0.222222,0.583333,0.333333,...,1299.33310,0.011749,0.0,306.15851,97.117401,289.31995,88.414680,10.804146,1332.40000,0
7,Calhoun,Alabama,6.413799,1,0.294118,0.240356,0.170886,0.182482,0.250000,0.317073,...,1722.61580,0.011815,0.0,305.10696,96.385132,286.03229,86.585365,12.306849,1170.50000,0
8,Chambers,Alabama,166.820390,1,0.236842,0.148148,0.142857,0.375000,0.444444,0.500000,...,1286.06680,0.008410,0.0,305.66736,96.237213,287.58121,87.767601,11.423082,1325.20000,0
9,Cherokee,Alabama,5.057791,1,0.227273,0.176471,0.083333,0.166667,0.363636,0.312500,...,1118.71250,0.007538,0.0,305.25671,96.038086,285.22256,87.757225,12.176334,1204.90000,0


# Question 2

In [6]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [7]:
filtered_data.describe()

,deathspc,intersects_msa,cur_smoke_q1,cur_smoke_q2,cur_smoke_q3,cur_smoke_q4,bmi_obese_q1,bmi_obese_q2,bmi_obese_q3,bmi_obese_q4,exercise_any_q1,exercise_any_q2,exercise_any_q3,exercise_any_q4,brfss_mia,puninsured2010,reimb_penroll_adj10,mort_30day_hosp_z,adjmortmeas_amiall30day,adjmortmeas_chfall30day,med_prev_qual_z,primcarevis_10,diab_hemotest_10,diab_eyeexam_10,diab_lipids_10,mammogram_10,cs00_seg_inc,cs00_seg_inc_pov25,cs00_seg_inc_aff75,cs_race_theil_2000,gini99,poor_share,inc_share_1perc,frac_middleclass,scap_ski90pcm,rel_tot,cs_frac_black,cs_frac_hisp,unemp_rate,cs_labforce,cs_elf_ind_man,cs_born_foreign,mig_inflow,mig_outflow,pop_density,frac_traveltime_lt15,hhinc00,median_house_value,ccd_exp_tot,score_r,cs_fam_wkidsinglemom,subcty_exp_pc,taxrate,tax_st_diff_top20,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax,pm25,bmcruderate,pm25_mia
count,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3103.000000,3106.000000,3106.000000,3107.000000,3012.000000,3098.000000,3069.000000,3054.000000,3057.000000,3029.000000,3107.000000,3107.000000,3107.000000,3107.000000,3008.000000,3107.000000,3008.000000,3106.000000,3107.000000,3106.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3017.000000,3017.000000,3107.000000,3107.000000,3107.000000,3.107000e+03,3080.000000,3069.000000,3107.000000,3107.000000,3107.000000,3106.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.00000,3107.000000
mean,23.790131,0.596717,0.212659,0.171048,0.134467,0.098316,0.239166,0.214580,0.209621,0.186739,0.455995,0.555671,0.603792,0.638727,0.249437,18.469460,9302.737743,0.457806,0.165483,0.108969,-0.148547,80.865348,83.706025,66.080221,78.307420,63.110073,0.025892,0.024278,0.026463,0.075402,0.379021,0.141739,0.094808,0.554244,0.000182,53.224564,8.744503,6.209190,0.049871,0.609344,0.159118,3.441958,0.028677,0.027522,244.325026,0.403803,32853.502978,1.121801e+05,6.092697,0.077348,0.194598,2119.407531,0.023089,0.775634,303.126997,88.970517,280.404875,87.469432,8.371871,1029.15597,0.003540
std,67.852145,0.490636,0.149348,0.128130,0.132181,0.110110,0.165928,0.153237,0.175849,0.167227,0.273874,0.322336,0.357861,0.376922,0.432757,5.536651,1590.926253,1.206493,0.039408,0.023565,0.863881,7.401457,6.594153,7.598549,7.854145,8.397699,0.030576,0.030757,0.032920,0.084131,0.086677,0.065460,0.050631,0.093099,1.347960,18.502524,14.483719,12.050404,0.017738,0.070393,0.090862,4.836270,0.019034,0.013780,1676.096088,0.137215,6975.837500,6.318905e+04,2.103573,9.007980,0.067828,999.833466,0.013848,1.470989,3.173950,9.689271,6.597855,4.811207,2.565927,248.38181,0.059405
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.625483,3663.530000,-7.778000,0.000000,0.000000,-4.853847,18.331749,16.911765,31.372549,19.661336,30.000000,-0.013363,-0.019502,-0.001993,0.000000,0.160954,0.000000,0.018570,0.215630,-4.258739,1.816347,0.000000,0.082034,0.016092,0.319209,0.000000,0.000000,0.000000,0.000000,0.099542,0.099878,10511.805000,0.000000e+00,3.032457,-38.687138,0.024793,0.000000,0.000000,0.000000,290.455540,31.643282,264.693820,58.159798,0.000000,189.30000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080128,0.000000,0.000000,0.000000,0.312500,0.444444,0.354167,0.400000,0.000000,14.410248,8159.340000,-0.255867,0.145312,0.096301,-0.615591,78.803466,81.108462,61.258165,75.000000,57.943925,0.005047,0.004164,0.003455,0.015591,0.317518,0.095383,0.062577,0.491883,-0.964225,39.669796,0.264501,0.917235,0.037422,0.567037,0.088637,0.898505,0.016502,0.018767,17.479568,0.299927,28733.524500,7.704740e+04,5.027049,-4.969633,0.152436,1510.192750,0.014993,0.000000,300.848035,88.052494,275.113020,85.093342,6.309710,864.29999,0.000000
50%,3.802303,1.000000,0.250000,0.198718,0.142857,0.096535,0.272076,0.24159

# Question 3

In [8]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3107 entries, 0 to 3106
Data columns (total 63 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   county                   3107 non-null   object 
 1   state                    3107 non-null   object 
 2   deathspc                 3107 non-null   float64
 3   intersects_msa           3107 non-null   int64  
 4   cur_smoke_q1             3107 non-null   float64
 5   cur_smoke_q2             3107 non-null   float64
 6   cur_smoke_q3             3107 non-null   float64
 7   cur_smoke_q4             3107 non-null   float64
 8   bmi_obese_q1             3107 non-null   float64
 9   bmi_obese_q2             3107 non-null   float64
 10  bmi_obese_q3             3107 non-null   float64
 11  bmi_obese_q4             3107 non-null   float64
 12  exercise_any_q1          3107 non-null   float64
 13  exercise_any_q2          3107 non-null   float64
 14  exercise_any_q3         

In [9]:
filtered_data.dropna(inplace=True)

In [10]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2915 entries, 0 to 3106
Data columns (total 63 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   county                   2915 non-null   object 
 1   state                    2915 non-null   object 
 2   deathspc                 2915 non-null   float64
 3   intersects_msa           2915 non-null   int64  
 4   cur_smoke_q1             2915 non-null   float64
 5   cur_smoke_q2             2915 non-null   float64
 6   cur_smoke_q3             2915 non-null   float64
 7   cur_smoke_q4             2915 non-null   float64
 8   bmi_obese_q1             2915 non-null   float64
 9   bmi_obese_q2             2915 non-null   float64
 10  bmi_obese_q3             2915 non-null   float64
 11  bmi_obese_q4             2915 non-null   float64
 12  exercise_any_q1          2915 non-null   float64
 13  exercise_any_q2          2915 non-null   float64
 14  exercise_any_q3         

# Question 4

In [11]:
filtered_data = pd.get_dummies(filtered_data, columns=['state'])

In [12]:
filtered_data.head()

,county,deathspc,intersects_msa,cur_smoke_q1,cur_smoke_q2,cur_smoke_q3,cur_smoke_q4,bmi_obese_q1,bmi_obese_q2,bmi_obese_q3,bmi_obese_q4,exercise_any_q1,exercise_any_q2,exercise_any_q3,exercise_any_q4,brfss_mia,puninsured2010,reimb_penroll_adj10,mort_30day_hosp_z,adjmortmeas_amiall30day,adjmortmeas_chfall30day,med_prev_qual_z,primcarevis_10,diab_hemotest_10,diab_eyeexam_10,diab_lipids_10,mammogram_10,cs00_seg_inc,cs00_seg_inc_pov25,cs00_seg_inc_aff75,cs_race_theil_2000,gini99,poor_share,inc_share_1perc,frac_middleclass,scap_ski90pcm,rel_tot,cs_frac_black,cs_frac_hisp,unemp_rate,cs_labforce,cs_elf_ind_man,cs_born_foreign,mig_inflow,mig_outflow,pop_density,frac_traveltime_lt15,hhinc00,median_house_value,ccd_exp_tot,score_r,cs_fam_wkidsinglemom,subcty_exp_pc,taxrate,tax_st_diff_top20,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax,pm25,bmcruderate,pm25_mia,state_Alabama,state_Arizona,state_Arkansas,state_California,state_Colorado,state_Connecticut,state_Delaware,state_Florida,state_Georgia,state_Idaho,state_Illinois,state_Indiana,state_Iowa,state_Kansas,state_Kentucky,state_Louisiana,state_Maine,state_Maryland,state_Massachusetts,state_Michigan,state_Minnesota,state_Mississippi,state_Missouri,state_Montana,state_Nebraska,state_Nevada,state_New Hampshire,state_New Mexico,state_New York,state_North Carolina,state_North Dakota,state_Ohio,state_Oklahoma,state_Oregon,state_Pennsylvania,state_Rhode Island,state_South Carolina,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,Autauga,16.548864,1,0.333333,0.238095,0.208333,0.133333,0.375000,0.238095,0.260870,0.133333,0.500000,0.666667,0.666667,0.800000,0,13.601278,9489.02,0.799735,0.146564,0.111778,0.574764,85.528270,84.771574,72.419628,83.925550,68.465909,0.036455,0.043313,0.037231,0.128216,0.379976,0.109228,0.06143,0.51950,-0.897834,56.112755,17.008999,1.396808,0.037379,0.651493,0.164787,1.165533,0.057040,0.051018,73.277412,0.233124,34379.539,126368.4,4.460057,-7.308133,0.191595,1059.66930,0.011183,0.0,306.02344,96.055420,288.08508,85.651848,11.712587,859.29999,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Baldwin,8.959118,1,0.268097,0.233503,0.167464,0.176991,0.298050,0.262467,0.193237,0.135747,0.599432,0.748677,0.835000,0.837963,0,19.085325,9618.34,-0.113602,0.145558,0.107229,0.118566,83.215372,80.871050,64.047822,78.992314,69.873998,0.032571,0.021566,0.041103,0.100549,0.489738,0.101471,0.12719,0.49911,-0.362414,47.242817,10.224691,1.756223,0.039112,0.598249,0.125441,2.105901,0.044860,0.023537,87.960236,0.295529,39219.598,163292.5,4.596590,-13.628747,0.186778,2209.91040,0.011756,0.0,305.51663,97.971542,290.20886,89.730972,10.077723,976.20001,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Barbour,6.609756,0,0.228571,0.250000,0.181818,0.111111,0.294118,0.571429,0.545455,0.277778,0.542857,0.571429,0.909091,0.722222,0,18.513805,9761.77,0.552571,0.169922,0.107575,0.067363,88.866842,85.831622,67.556468,81.519507,64.768683,0.021680,0.021777,0.020898,0.039993,0.490637,0.267998,0.09734,0.40833,-1.499349,41.170193,46.039669,1.646119,0.068132,0.480174,0.313745,1.498037,0.016565,0.023073,32.814877,0.394128,24274.195,91443.8,4.734407,-15.955111,0.337853,1570.24830,0.012059,0.0,306.06226,97.371674,289.24210,88.633575,10.981967,1040.90000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bibb,6.038192,1,0.244444,0.280000,0.181818,0.150000,0.466667,0.375000,0.190476,0.100000,0.422222,0.560000,0.681818,0.750000,0,17.718826,11269.81,1.857274,0.234408,0.112190,-0.235594,83.723878,85.478548,64.356436,80.528053,57.228916,0.017462,0.028094,0.002048,0.103550,0.417098,0.206075,0.06622,0.46136,-0.930472,46.365120,22.010948,1.008355,0.061639,0.528893,0.237480,0.427350,0.035821,0.029338,33.427227,0.246582,24927.521,99441.8,4.154157,-14.839100,0.197729,1338.44

# Question 5

In [13]:
X = filtered_data.drop(['deathspc','county'], axis=1)
y = filtered_data['deathspc']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, 
random_state=10)

# Question 6(a)

In [15]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [16]:
X_norm = scaler.transform(X_train)

In [17]:
reg = LinearRegression().fit(X_norm, y_train)

In [18]:
print(reg.score(X_norm, y_train))
print(reg.score(X_test, y_test))

0.43199780957930267
-1.0708290388517746e+23


In [19]:
y_pred = reg.predict(X_test)
mean_squared_error(y_test, y_pred)

1.7131906369459238e+26

# Question 6(b)

Yes there is concern for overfitting. This is due to how low the MSE is for the training set (0.43199780957930267) but it instantly becomes extremely high for the test set (1.7131906369459238e+26). While we do expect the MSE to be higher for the test set than the training set, in this situation, the training MSE is a fraction while the test MSE is over septillion. This huge increase is suspicious and points to possible overfitting. 

Links used:

https://stackoverflow.com/questions/58740329/patsyerror-number-of-rows-mismatch-between-data-argument-and-column-statsmodel

https://stackoverflow.com/questions/29586323/how-to-retain-column-headers-of-data-frame-after-pre-processing-in-scikit-learn

# Question 7(a)(b)(c)

In [ ]:
# Lasso
kf = KFold(n_splits=10, random_state = 25, shuffle=True)
tkf = kf.split(X, y)
lasso = Lasso()
alphacl = (np.linspace(0, 1, 101))
alpha_lgrid = [{'alpha': alphacl}]
def vector_values(grid_search, trials):
    mean_vec = np.zeros(trials)
    std_vec = np.zeros(trials)
    i = 0
    final = grid_search.cv_results_
    
   
    for mean_score, std_score in zip(final["mean_test_score"], 
final["std_test_score"]):
        mean_vec[i] = -mean_score
        std_vec[i] = std_score
        i = i+1
    return mean_vec, std_vec
grid_search_lasso = GridSearchCV(lasso, alpha_lgrid, cv = tkf, scoring = 
'neg_mean_squared_error')
grid_search_lasso.fit(X, y)
mean_vec, std_vec = vector_values(grid_search_lasso, 101)
results_cv_lasso = pd.DataFrame({"alphas": alphacl, "MSE": mean_vec,})
min_mse = min(results_cv['MSE'])
results_cv_lasso.loc[results_cv['MSE']==min_mse]
plt.plot(
    alphacl,
    mean_vec,
    linewidth=2,
)

# Ridge 
kf = KFold(n_splits=10, random_state = 25, shuffle=True)
tkf = kf.split(X, y)
ridge = Ridge()
alphacr = (np.linspace(0, 25, 251))
alpha_rgrid = [{'alpha': alphacr}]
grid_search_ridge = GridSearchCV(ridge, alpha_rgrid, cv = tkf, scoring = 
'neg_mean_squared_error')
grid_search_ridge.fit(X, y)
mean_vec, std_vec = vector_values(grid_search_ridge, 251)
results_cv_ridge = pd.DataFrame(
    {
        "alphas": alphacr,
        "MSE": mean_vec,
    }
)
min_mse = min(results_cv['MSE'])
results_cv_ridge.loc[results_cv['MSE']==min_mse]
plt.plot(
    alphacr,
    mean_vec,
    linewidth=2,
)

# Question 7(d)

In [ ]:
results_cv_lasso.loc[results_cv['MSE']==min_mse]

In [ ]:
results_cv_ridge.loc[results_cv['MSE']==min_mse]

# Question 7(e)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_norm = scaler.transform(X_train)

In [ ]:
ols_lasso = Lasso(alpha=0.23)
ols_lasso.fit(X_norm, y_train)
ols_ridge = Ridge(alpha=2.3)
ols_ridge.fit(X_norm, y_train)

# Question 8

In [ ]:
y_pred_lasso = ols_lasso.predict(X_test)
mean_squared_error(y_test, y_pred_lasso)

In [ ]:
y_pred_ridge = ols_ridge.predict(X_test)
mean_squared_error(y_test, y_pred_ridge)

Both lasso and ridge lowered the MSE by over half of what it was with OLS with lasso performing the best.  

To be perfectly honest, none of these models are performing well. This could be because of the random state we are using although it is unclear without redoing all of this analysis with multiple different random states. But if I was forced to choose, I would choose lasso. First off, since OLS estimates have such a high variance in this situation, ridge and lasso would be better off. While ridge tends to perform better when the outcome is a function of relatively many predictors and lasso tends is better when the outcome is a function of relatively few predictors, that is a general statement. We will always need to test this to determine which approach works better and when we did lasso performs significantly better.